In [2]:
!pip install yfinance requests pandas numpy matplotlib statsmodels scipy pmdarima --quiet

In [22]:
import yfinance as yf
import pandas as pd

btc = yf.download(
    "BTC-USD",
    period="2y",
    progress=False,
    auto_adjust=False       
)["Close"]

btc.to_csv("btc_historical.csv")

In [37]:
url = "https://api.elections.kalshi.com/trade-api/v2/markets"
params = {"limit": 1000, "status": "open"}
response = requests.get(url, params=params)
kalshi_strikes = []

if response.status_code == 200 and any("BTC" in m.get("ticker","") for m in response.json().get("markets",[])):
    print("API gave data")
else:
    print("API no data, using backup Kalshi strikes from Nov. 18")

kalshi_strikes = [
    (91000,  0.98),
    (91250,  0.93),
    (91500,  0.70),
    (91750,  0.41),
    (92000,  0.17),
    (92250,  0.07),
    (92500,  0.05)]

API no data, using backup Kalshi strikes from Nov. 18


In [38]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np

log_btc = np.log(btc)
model = SARIMAX(log_btc, order=(5,1,0), seasonal_order=(1,1,1,7), enforce_stationarity=False)
fitted = model.fit(disp=False)
forecast = fitted.get_forecast(steps=1)

pred    = np.exp(forecast.predicted_mean.iloc[0])
ci_low  = np.exp(forecast.conf_int().iloc[0,0])
ci_high = np.exp(forecast.conf_int().iloc[0,1])

print(f"\nPREDICTION FOR NOV 19 (1 PM ET CLOSE)")
print(f"→ Point forecast : ${pred:,.0f}")
print(f"→ 95% interval   : ${ci_low:,.0f} – ${ci_high:,.0f}")

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)



PREDICTION FOR NOV 19 (1 PM ET CLOSE)
→ Point forecast : $90,697
→ 95% interval   : $86,217 – $95,410


In [39]:
from scipy.stats import norm

std = (ci_high - ci_low) / (2 * 1.96)

print("\nBITCOIN BET RECOMMENDATIONS – NOV 19 1PM ET CLOSE")
print("="*85)
print(f"{'Strike':>8}  {'Model Prob':>10}  {'Market Prob':>11}  {'Edge':>8}  Recommendation")
print("-"*85)

for strike, market_prob in kalshi_strikes:
    model_prob = 1 - norm.cdf(strike, pred, std)
    edge = model_prob - market_prob
    rec = "BET YES" if edge > 0.03 else "BET NO" if edge < -0.03 else "NO EDGE"
    print(f"${strike:>6,}    {model_prob:>8.1%}      {market_prob:>8.1%}    {edge:+6.1%}    {rec}")


BITCOIN BET RECOMMENDATIONS – NOV 19 1PM ET CLOSE
  Strike  Model Prob  Market Prob      Edge  Recommendation
-------------------------------------------------------------------------------------
$91,000       44.9%         98.0%    -53.1%    BET NO
$91,250       40.7%         93.0%    -52.3%    BET NO
$91,500       36.6%         70.0%    -33.4%    BET NO
$91,750       32.7%         41.0%     -8.3%    BET NO
$92,000       28.9%         17.0%    +11.9%    BET YES
$92,250       25.4%          7.0%    +18.4%    BET YES
$92,500       22.1%          5.0%    +17.1%    BET YES
